### Hyper-parameter Tunning with Genetic Algorithm
#### Created: Dec. 29 2018

In [1]:
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os

In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [3]:
import core.genetic.control as control
import core.genetic.tuner as tuner
import core.models.stacked_lstm as stacked_lstm

In [4]:
# Load dataset.
print("Dataset paths saved:")
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]
print(f"Dataset chosen: {FILE_DIR}")

Dataset paths saved:
{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> a
Dataset chosen: /Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv


In [ ]:
# Load configuration.
print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    print("\t" + cf)

config_name = input("Select config file >>> ")
if config_name.endswith(".py"):
    config_name = config_name[:-3]
# config_name = "mac_config"

exec(f"import hps_configs.{config_name} as config")

Avaiable configuration files found: 
	mac_lite_config.py
	ec2_config.py
	__pycache__
	ec2_lite_config.py
	mac_config.py
	template.py
Select config file >>> mac_lite_config


In [ ]:
# Training Spec.
GENE_POOL = config.main
TOTAL_GEN = 3
POP_SIZE = 10
IGNORE_SET = (
    "PERIODS", "ORDER", "TRAIN_RATIO", "VAL_RATIO", "num_outputs", "num_inputs", "report_periods",
    "tensorboard_path", "model_path", "fig_path"
)

In [ ]:
def eval_net_wrapper(param):
    return control.eval_net(
        model=stacked_lstm.StackedLSTM,
        param=param,
        file_dir=FILE_DIR,
        metric="mse_val",
        smooth_metric=0.05
    )

In [ ]:
# Create the Optimizer.
optimizer = tuner.GeneticTuner(
    gene_pool=config.main,
    pop_size=POP_SIZE,
    eval_func=eval_net_wrapper,
    mode="min",
    retain=0.6,
    shot_prob=0.1,
    mutate_prob=0.7,
    verbose=False,
    ignore=IGNORE_SET
)

In [ ]:
start_time = datetime.datetime.now()

In [ ]:
elite = control.train_op(
    optimizer=optimizer,
    total_gen=TOTAL_GEN,
    elite=0.5,
    write_to_disk="/Users/tianyudu/Downloads/gen"
)

Generation: [0/3]
Evaluating the initial population.
 ########## Evaluating... [10/10, 100.00%] Current Net: size=[16, 32], ep=4, lr= 0.01003001000
Best fitted entity validatiton MSE:  0.0526742        
Worst fitted entity validation MSE:  1.2418854
Generation: [1/3]
 #####------ Evaluating... [5/11,  45.45%] Current Net: size=[18, 62], ep=3, lr= 1.0150= 1.2000

In [ ]:
end_time = datetime.datetime.now()
print(f"Time taken: {end_time - start_time}")

In [ ]:
# Report
print("Best found:")
print(list(elite.values())[-1][0])

In [ ]:
plt.close()
plt.style.use('seaborn-dark')
plt.title("The Elite Group")
plt.plot([
    elite_gp[0][1] 
    for elite_gp in list(elite.values())
    ]
)
plt.plot([
    np.mean([x[1] for x in elite_gp])
    for elite_gp in list(elite.values())
    ]
)
plt.plot([
    elite_gp[-1][1] 
    for elite_gp in list(elite.values())]
)
plt.legend(
    ["Best Perform. val.mse", "Average val. mse", "Worst Perform. val.mse"],
    loc="best"
)
plt.show()